# **This is simple implementation of agentic workflow using various LLM endpoints/providers with Langchain and Langraph**

Installing dependencies

In [1]:
!pip install langchain -qU
!pip install langgraph -qU
!pip install langchain-anthropic -qU
!pip install langchain-groq -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.5/951.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.8 MB/s eta 0:00:00


Importing dependencies

In [2]:
import nest_asyncio
from typing import List, TypedDict, Any, Tuple
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END, START
from datetime import date, datetime
from langgraph.checkpoint.memory import MemorySaver

nest_asyncio.apply()

# **API keys**

In [3]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDEAI_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
FINANCIAL_MODELING_PREP_API_KEY = userdata.get('FMP_API_KEY')

# **Functions**

In [4]:
def convert_to_dbl_qt(input: str) -> str:
  return input.replace("'", '"')

# get_stock_price

In [5]:
import os
import requests
from pprint import pprint
class StockPrice(BaseModel):
    symbol:str = Field(description="The symbol of the company")
    price:float = Field(description="The price of the company")
    volume:float = Field(description="The volume of the company")
    priceAvg50:float = Field(description="The 50 day average price of the company")
    priceAvg200:float = Field(description="The 200 day average price of the company")
    eps:float = Field(description="The EPS of the company")
    pe:float = Field(description="The PE of the company")
    earningsAnnouncement:datetime = Field(description="The earnings announcement of the company")
# Define the functions that will fetch financial data
def get_stock_price(symbol):
    """
    Fetch the current stock price for the given symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.
    """
    try:
      url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
      response = requests.get(url)
      data = response.json()
      stock_price = StockPrice(**data[0])
      return stock_price
    except (IndexError, KeyError):
        return {"error": f"Could not fetch price for symbol: {symbol}"}

## DATA PROVIDED BY THIS ENDPOINT:
# [{'symbol': 'AAPL',
#   'name': 'Apple Inc.',
#   'price': 222.5,
#   'changesPercentage': -0.1212,
#   'change': -0.27,
#   'dayLow': 221.91,
#   'dayHigh': 224.03,
#   'yearHigh': 237.23,
#   'yearLow': 164.08,
#   'marketCap': 3382912250000,
#   'priceAvg50': 223.0692,
#   'priceAvg200': 195.382,
#   'exchange': 'NASDAQ',
#   'volume': 35396922,
#   'avgVolume': 57548506,
#   'open': 223.58,
#   'previousClose': 222.77,
#   'eps': 6.57,
#   'pe': 33.87,
#   'earningsAnnouncement': '2024-10-31T00:00:00.000+0000',
#   'sharesOutstanding': 15204100000,
#   'timestamp': 1726257601}]

# get_company_financials

In [6]:
class CompanyFinancials(BaseModel):
    symbol:str =  Field(description="The symbol of the company")
    companyName:str =  Field(description="The name of the company")
    marketCap:float = Field(alias="mktCap", description="The market capitalization of the company")
    industry:str =  Field(description="The industry of the company")
    sector:str =  Field(description="The sector of the company")
    website:str =  Field(description="The website of the company")
    beta:float = Field(description="The beta of the company")
    price:float = Field(description="The price of the company")

def get_company_financials(symbol) -> Tuple[Any, CompanyFinancials]:
    """
    Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.
    """
    try:
      url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
      response = requests.get(url)
      data = response.json()
      financials = CompanyFinancials(**data[0])
      return financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

## DATA PROVIDED BY THIS ENDPOINT:
# [{'symbol': 'AAPL',
#   'price': 222.5,
#   'beta': 1.24,
#   'volAvg': 57548506,
#   'mktCap': 3382912250000,
#   'lastDiv': 1,
#   'range': '164.08-237.23',
#   'changes': -0.27,
#   'companyName': 'Apple Inc.',
#   'currency': 'USD',
#   'cik': '0000320193',
#   'isin': 'US0378331005',
#   'cusip': '037833100',
#   'exchange': 'NASDAQ Global Select',
#   'exchangeShortName': 'NASDAQ',
#   'industry': 'Consumer Electronics',
#   'website': 'https://www.apple.com',
#   'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original content; Apple Card, a co-branded credit card; and Apple Pay, a cashless payment service, as well as licenses its intellectual property. The company serves consumers, and small and mid-sized businesses; and the education, enterprise, and government markets. It distributes third-party applications for its products through the App Store. The company also sells its products through its retail and online stores, and direct sales force; and third-party cellular network carriers, wholesalers, retailers, and resellers. Apple Inc. was incorporated in 1977 and is headquartered in Cupertino, California.',
#   'ceo': 'Mr. Timothy D. Cook',
#   'sector': 'Technology',
#   'country': 'US',
#   'fullTimeEmployees': '161000',
#   'phone': '408 996 1010',
#   'address': 'One Apple Park Way',
#   'city': 'Cupertino',
#   'state': 'CA',
#   'zip': '95014',
#   'dcfDiff': 55.70546,
#   'dcf': 166.79453554058594,
#   'image': 'https://financialmodelingprep.com/image-stock/AAPL.png',
#   'ipoDate': '1980-12-12',
#   'defaultImage': False,
#   'isEtf': False,
#   'isActivelyTrading': True,
#   'isAdr': False,
#   'isFund': False}]

# get_income_statement

In [7]:
class IncomeStatement(BaseModel):
    date_field: date = Field(alias='date', description="The date of the income statement")
    revenue:float = Field(description="The revenue of the company")
    gross_profit:float = Field(alias='grossProfit', description="The gross profit of the company")
    net_income:float = Field(alias='netIncome', description="The net income of the company")
    ebitda:float = Field(description="The EBITDA of the company")
    eps:float = Field(description="The EPS of the company")
    eps_diluted:float = Field(alias='epsdiluted', description="The EPS diluted of the company")


def get_income_statement(symbol):
    """
    Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.
    """
    try:
      url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FINANCIAL_MODELING_PREP_API_KEY}"
      response = requests.get(url)
      data = response.json()
      financials = IncomeStatement(**data[0])
      return financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

## DATA PROVIDED BY THIS ENDPOINT:
# {'date': '2023-09-30',
#   'symbol': 'AAPL',
#   'reportedCurrency': 'USD',
#   'cik': '0000320193',
#   'fillingDate': '2023-11-03',
#   'acceptedDate': '2023-11-02 18:08:27',
#   'calendarYear': '2023',
#   'period': 'FY',
#   'revenue': 383285000000,
#   'costOfRevenue': 214137000000,
#   'grossProfit': 169148000000,
#   'grossProfitRatio': 0.4413112958,
#   'researchAndDevelopmentExpenses': 29915000000,
#   'generalAndAdministrativeExpenses': 0,
#   'sellingAndMarketingExpenses': 0,
#   'sellingGeneralAndAdministrativeExpenses': 24932000000,
#   'otherExpenses': 382000000,
#   'operatingExpenses': 55229000000,
#   'costAndExpenses': 269366000000,
#   'interestIncome': 3750000000,
#   'interestExpense': 3933000000,
#   'depreciationAndAmortization': 11519000000,
#   'ebitda': 125820000000,
#   'ebitdaratio': 0.3282674772,
#   'operatingIncome': 114301000000,
#   'operatingIncomeRatio': 0.2982141227,
#   'totalOtherIncomeExpensesNet': -565000000,
#   'incomeBeforeTax': 113736000000,
#   'incomeBeforeTaxRatio': 0.2967400237,
#   'incomeTaxExpense': 16741000000,
#   'netIncome': 96995000000,
#   'netIncomeRatio': 0.2530623426,
#   'eps': 6.16,
#   'epsdiluted': 6.13,
#   'weightedAverageShsOut': 15744231000,
#   'weightedAverageShsOutDil': 15812547000,
#   'link': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/0000320193-23-000106-index.htm',
#   'finalLink': 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000106/aapl-20230930.htm'}

# **Generate Report**

In [33]:
def generate_markdown_financials(company_financials: CompanyFinancials) -> str:
  return f"""

      ## Company Overview
      - **Name**: {company_financials.companyName}
      - **Symbol**: {company_financials.symbol}
      - **Market Capitalization**: {company_financials.marketCap}
      - **Industry**: {company_financials.industry}
      - **Sector**: {company_financials.sector}
      - **Website**: [{company_financials.website}]({company_financials.website})
      - **Beta**: {company_financials.beta: .3f}
      - **Current Price**: ${company_financials.price: .2f}
      """ if (company_financials) else " No company financials were obtained"

def generate_markdown_income_statement(income_statement: IncomeStatement) -> str:
  return f"""
    ## Income Statement (as of {income_statement.date_field})
    - **Revenue**: ${income_statement.revenue: .2f}
    - **Gross Profit**: ${income_statement.gross_profit: .2f}
    - **Net Income**: ${income_statement.net_income: .2f}
    - **EBITDA**: ${income_statement.ebitda: .2f}
    - **EPS**: {income_statement.eps: .2f}
    - **EPS (Diluted)**: {income_statement.eps_diluted: .2f}
    """ if income_statement else "No income statement was obtained"

def generate_markdown_stock_price(stock_price: StockPrice) -> str:
  return  f"""
      ## Stock Price Information
    - **Current Price**: ${stock_price.price: .2f}
    - **Volume**: {stock_price.volume: .2f}
    - **50-Day Average Price**: ${stock_price.priceAvg50: .2f}
    - **200-Day Average Price**: ${stock_price.priceAvg200: .2f}
    - **EPS**: {stock_price.eps: .2f}
    - **PE Ratio**: {stock_price.pe: .2f}
    - **Earnings Announcement**: {stock_price.earningsAnnouncement}
    """ if stock_price else "No stock price information was obtained"

def generate_markdown_report(company_financials: str, income_statement: str, stock_price: str) -> str:
    """
    Generates a markdown report from the GraphState instance.
    """
    md_report = f"""

    {company_financials}

    {income_statement}

    {stock_price}
    """
    return md_report

# **Anthropic LLM**

In [9]:
# from langchain_anthropic import ChatAnthropic
# llm = ChatAnthropic(api_key=CLAUDE_API_KEY, model_name='claude-3-sonnet-20240229', temperature=0.0)

# **GROQ LLM**

In [10]:
from langchain_groq import ChatGroq

llm = ChatGroq(api_key=GROQ_API_KEY,
    # model="mixtral-8x7b-32768",
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# **Router Chain**

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
from typing_extensions import Literal
class RouterResult(BaseModel):
  route:Literal['income_statement', 'report', 'company_financials', 'stock_price', "chat"] = Field(description="LLM's decision the route to take")

In [13]:
# system_route = f"""You are very helpful Financial assistant. Per user's prompt you need to decide user's intention. your response can be only following values:
#              - income_statement,
#              - report,
#              - company_financials,
#              - stock_price,
#              - chat

#             When you do not know or not sure use chat.

#             Example of request
#             What is the stock price of Apple?
#             stock_price

#             Example of request
#             What is/Tell me/Give me/Show me the income statement of Apple?
#             What is grossProfit of Apple?

#             income_statement

#             Example of request
#             What is the company financials of Apple?
#             company_financials

#             Example of request
#             Tell me about Apple?
#             report


system_route = """
You are a highly knowledgeable Financial Assistant, designed to analyze and categorize user requests based on their intent. Choose from the following categories, and respond only with one of these values:

- **income_statement**
- **report**
- **company_financials**
- **stock_price**
- **chat**

If the intent isn't clear or doesn't match any specific category, use **chat**.

**Examples:**

1. **Stock Price Requests**
   - "What is the stock price of Apple?"
     **stock_price**
   - "Show me the latest price for Microsoft."
     **stock_price**

2. **Income Statement Requests**
   - "What is the income statement of Apple?"
     **income_statement**
   - "Give me the gross profit of Tesla for last year."
     **income_statement**

3. **Company Financials Requests**
   - "What are the financials for Google?"
     **company_financials**
   - "Show me Apple’s financial position."
     **company_financials**

4. **Report Requests**
   - "Tell me about Apple's business."
     **report**
   - "Can you provide an overview of Amazon?"
     **report**

5. **Chat Requests (unclear or conversational)**
   - "What do you think of Apple?"
     **chat**
   - "Any thoughts on the tech market?"
     **chat**

"""
route_prompt = ChatPromptTemplate.from_messages(
     [
        ("system", system_route),
        ("human", "{request}"),
    ]
)

In [14]:
def create_route_chain(llm):
  return route_prompt | llm.with_structured_output(RouterResult)

In [15]:
route_chain_temp = create_route_chain(llm)
print(route_chain_temp.invoke({"request": "proved overview of uber?"}))

route='report'


# **Chat chain**

In [16]:
class ChatResult(BaseModel):
  response: str = Field(description="LLM's response")

In [17]:
system_chat = """
You are a very helpful Assistant, you are to answer user's request to the best of your ability. if you do not know respond with I do not know.
"""

chat_prompt = ChatPromptTemplate.from_messages(
     [
        ("system", system_chat),
        ("human", "{request}"),
    ]
)

In [18]:
def create_chat_chain(llm):
  return chat_prompt | llm.with_structured_output(ChatResult)

In [19]:
chat_chain_temp = create_chat_chain(llm)
print(chat_chain_temp.invoke({"request": "How is the weather?"}))

response='I don’t have real-time access to current weather conditions. However, I can suggest some ways for you to find out the current weather. You can check online weather websites such as AccuWeather, Weather.com, or the National Weather Service for the most up-to-date information. Alternatively, you can ask a voice assistant like Siri, Google Assistant, or Alexa to provide you with the current weather conditions.'


# **Generation Chain**

In [20]:
class Extraction(BaseModel):
  symbol: str = Field(description="The symbol of the company")


In [21]:
UNKNOWN = 'UNKNOWN'
system_extraction = f"""You are very helpful Financial assistant.User will request financial data/information for a company.You are to return company's symbol on a stock market.
            do not make anything up. if you do not know reply {UNKNOWN}.
"""

extraction_prompt = ChatPromptTemplate.from_messages(
     [
        ("system", system_extraction),
        ("human", "{request}"),
    ]
)

In [22]:
def create_extraction_chain(llm):
  return extraction_prompt | llm.with_structured_output(Extraction)

# **Testing extraction chain**

In [23]:
extraction_chain_temp = create_extraction_chain(llm)
result: Extraction = extraction_chain_temp.invoke({"request": "What is the stock price of Apple?"})
print(result.symbol)

AAPL


# **Graph State**

In [24]:
class GraphState(TypedDict):
  """
     Represents the state of our graph.

    Attributes:
        symbol: The symbol of the company.
        income_statement: The income statement of the company.
        company_financials: The company financials of the company.
        stock_price: The stock price of the company.
  """
  symbol: str
  request: str
  income_statement: str
  company_financials: str
  stock_price: str
  extraction_chain: Any
  chat_chain: Any
  report_md: str
  final_answer: str
  request_category: str
  chat_response: str
  error: str

In [25]:
from langgraph.graph import END, StateGraph

# **Graph Nodes**

In [26]:
# def save_md_report_to_file(md_report: str, filename: str = "financial_report.md"):
#   """Saves the provided markdown report to a file."""
#   with open(filename, "w") as f:
#     f.write(md_report)
#   print(f"Markdown report saved to {filename}")

In [54]:
def create_get_route_node(llm):
  print('create_get_route_node')
  chain = create_route_chain(llm)

  def get_route_node(state: GraphState):
    print('router_node')
    print('State', state)
    result: RouterResult = chain.invoke({"request": state['request']})
    print('request_category:', result.route)
    # state['request_category'] = result.route
    return {'request_category':result.route}
  return get_route_node

def create_symbol_extraction_node(llm):
  print('create_symbol_extraction_node')
  chain = create_extraction_chain(llm)
  def symbol_extraction_node(state: GraphState):
    print('extraction_node')
    print('State', state)
    symbol  = UNKNOWN
    try:
      # chain = state['extraction_chain']
      # if chain is None:
      #   raise Exception('Extraction chain is not initialized')

      result: Extraction = chain.invoke(state['request'])
      # result: Extraction = extraction_chain_p.invoke(state['request'])
      symbol = result.symbol
    except Exception as e:
      print('Error:', e)

    # print('Symbol:', state['symbol'])
    return {'symbol': symbol}
  return symbol_extraction_node

def get_income_statement_node(state: GraphState):
  print('get_income_statement_node')
  print('Symbol:', state['symbol'])
  income_statement: IncomeStatement = get_income_statement(state['symbol'])
  result = generate_markdown_income_statement(income_statement)
  return {'income_statement': result}

def get_company_financials_node(state: GraphState):
  print('get_company_financials_node')
  print('Symbol:', state['symbol'])
  info: CompanyFinancials = get_company_financials(state['symbol'])
  result = generate_markdown_financials(info)
  return {'company_financials': result}

def get_stock_price_node(state: GraphState):
  print('get_stock_price_node')
  print('Symbol:', state['symbol'])
  stock_price: StockPrice = get_stock_price(state['symbol'])
  result = generate_markdown_stock_price(stock_price)
  return {'stock_price': result}

def error_node(state: GraphState) -> str:
    return {'error':f"""
    Unknown Symbol: {state['symbol']}
    Can not produce report for this symbol.
    """}

def generate_markdown_report_node(state: GraphState) -> str:
    """
    Generates a markdown report from the GraphState instance.
    """
    company_financials = state['company_financials'] if 'company_financials' in state else None
    income_statement = state['income_statement'] if ('income_statement' in state) else None
    stock_price = state['stock_price'] if ('stock_price' in state) else None
    md_report = generate_markdown_report(company_financials=company_financials, income_statement=income_statement, stock_price=stock_price)
    file_name = f"{state['symbol']}_financial_report.md"
    # save_md_report_to_file(md_report, filename= file_name)
    return {'report_md': md_report}

def create_chain_node(llm):
  print('create_chain_node')
  chain = create_chat_chain(llm)
  def chat_node(state: GraphState) -> str:
      """
      Generates a response from llm per user request/prompt.
      """
      # chain = state['chat_chain']
      # if chain is None:
      #   raise Exception('Extraction chain is not initialized')

      result: ChatResult = chain.invoke({"request": state['request']})
      return {'chat_response': result.response}
  return chat_node

def final_answer_node(state: GraphState):
  print('final_answer_node')
  print('State', state)

  if state['request_category'] == 'income_statement':
    if  'error' in state:
      result = state['error']
    else:
      result = state['income_statement']
      result = f"# income_statement for ({state['symbol']}) \n" + result
  elif state['request_category'] == 'company_financials':
    if  'error' in state:
      result = state['error']
    else:
      result = state['company_financials']
      result = f"# company_financials for ({state['symbol']}) \n" + result
  elif state['request_category'] == 'stock_price':
    if  'error' in state:
      result = state['error']
    else:
      result = state['stock_price']
      result = f"# stock_price for ({state['symbol']}) \n" + result
  elif state['request_category'] == 'chat':
    result = state['chat_response']
  elif state['request_category'] == 'report':
    if 'report_md' in state:
      result = state['report_md']
      result = f"# Report for ({state['symbol']}) \n" + result
    elif 'error' in state:
      result = state['error']
    else:
      result = "Can not provide an answer"
  else:
    result = "Can not provide an answer"

  return {'final_answer':result}

In [50]:
def is_there_symbol(state: GraphState):
  print('is_there_symbol')
  print('State', state)
  if state['symbol']  == UNKNOWN:
    print('Symbol:', UNKNOWN)
    return False

  return True

def where_to_alone(state: GraphState):
  print('where_to_alone')
  print('State', state)
  if state['symbol'].upper()  == UNKNOWN:
    # print('Symbol:', UNKNOWN)
    print('Routing to error')
    return 'error'
  else:
    print(f"Routing to {state['request_category']}")
    return state['request_category']

  return True
def where_to(state: GraphState):
  print('where_to')
  print('State', state)
  if state['request_category'] == 'report':
    return 'report'
  elif state['request_category'] == 'chat':
    return 'chat'

  return 'alone'

In [55]:
ROUTER = 'Router'
SYMBOL_EXTRACTION_REPORT = 'SymbolExtractionReport'
SYMBOL_EXTRACTION_ALONE = 'SymbolExtractionAlone'
STOCK_PRICE = 'StockPrice'
INCOME_STATEMENT ='IncomeStatement'
COMPANY_FINANCIALS ='CompanyFinancials'
STOCK_PRICE_STAND_ALONE = 'StockPriceStandAlone'
INCOME_STATEMENT_STAND_ALONE ='IncomeStatementStandAlone'
COMPANY_FINANCIALS_STAND_ALONE ='CompanyFinancialsStandAlone'
FINAL_ANSWER = 'FinalAnswer'
# START_REPORT = 'StartReport'

CHAT_NODE = 'ChatNode'
ERROR_NODE='ErrorNode'
GENERATE_REPORT = 'GenerateReport'
PASS = 'Pass'

def create_workflow(llm):
  workflow = StateGraph(GraphState)
  workflow.add_node(ROUTER, create_get_route_node(llm))
  # # DEBUG
  # workflow.add_edge(ROUTER, END)
  # workflow.set_entry_point(ROUTER)
  # # END DEBUG

  workflow.add_node(SYMBOL_EXTRACTION_REPORT, create_symbol_extraction_node(llm))
  workflow.add_node(PASS, lambda state: state)

  workflow.add_node(INCOME_STATEMENT, get_income_statement_node)
  workflow.add_node(COMPANY_FINANCIALS, get_company_financials_node)
  workflow.add_node(STOCK_PRICE, get_stock_price_node)

  workflow.add_node(SYMBOL_EXTRACTION_ALONE, create_symbol_extraction_node(llm))
  workflow.add_node(INCOME_STATEMENT_STAND_ALONE, get_income_statement_node)
  workflow.add_node(COMPANY_FINANCIALS_STAND_ALONE, get_company_financials_node)
  workflow.add_node(STOCK_PRICE_STAND_ALONE, get_stock_price_node)
  workflow.add_node(CHAT_NODE, create_chain_node(llm))
  workflow.add_node(FINAL_ANSWER, final_answer_node)

  workflow.add_node(GENERATE_REPORT, generate_markdown_report_node)
  workflow.add_node(ERROR_NODE, error_node)
  workflow.set_entry_point(ROUTER)

  workflow.add_conditional_edges(ROUTER, where_to,  path_map={
      'report': SYMBOL_EXTRACTION_REPORT,
      'alone': SYMBOL_EXTRACTION_ALONE,
      'chat': CHAT_NODE
  })

  workflow.add_conditional_edges(SYMBOL_EXTRACTION_REPORT, is_there_symbol, {True:PASS, False: ERROR_NODE})
  workflow.add_conditional_edges(SYMBOL_EXTRACTION_ALONE, where_to_alone, {
      'error': ERROR_NODE,
      'income_statement': INCOME_STATEMENT_STAND_ALONE,
      'company_financials': COMPANY_FINANCIALS_STAND_ALONE,
      'stock_price': STOCK_PRICE_STAND_ALONE,
      })

  # workflow.add_conditional_edges(ROUTER, lambda x: x['request_category'],  path_map={
  #     'report': SYMBOL_EXTRACTION_REPORT,
  #     'income_statement': INCOME_STATEMENT_STAND_ALONE,
  #     'company_financials': COMPANY_FINANCIALS_STAND_ALONE,
  #     'stock_price': STOCK_PRICE_STAND_ALONE,
  #     'chat': CHAT_NODE
  # })


  # workflow.add_conditional_edges(SYMBOL_EXTRACTION_REPORT, is_there_symbol, {True:PASS, False: ERROR_NODE})

  workflow.add_edge(PASS,INCOME_STATEMENT)
  workflow.add_edge(PASS,COMPANY_FINANCIALS)
  workflow.add_edge(PASS,STOCK_PRICE)

  workflow.add_edge(INCOME_STATEMENT,GENERATE_REPORT)
  workflow.add_edge(COMPANY_FINANCIALS,GENERATE_REPORT)
  workflow.add_edge(STOCK_PRICE,GENERATE_REPORT)

  workflow.add_edge(GENERATE_REPORT, FINAL_ANSWER)
  workflow.add_edge(ERROR_NODE, FINAL_ANSWER)

  workflow.add_edge(INCOME_STATEMENT_STAND_ALONE,FINAL_ANSWER)
  workflow.add_edge(COMPANY_FINANCIALS_STAND_ALONE,FINAL_ANSWER)
  workflow.add_edge(STOCK_PRICE_STAND_ALONE,FINAL_ANSWER)
  workflow.add_edge(CHAT_NODE,FINAL_ANSWER)
  workflow.add_edge(FINAL_ANSWER, END)

  memory = MemorySaver()

  app = workflow.compile(checkpointer=memory)

  app.get_graph().draw_mermaid_png(output_file_path="financial_asstant_graph.png")
  return app
# app.nodes[EXTRACTION] = app.nodes[EXTRACTION].bind(extraction_chain_p=extraction_chain)
# app.nodes[EXTRACTION].func.__closure__[0].cell_contents = extraction_chain
# app.nodes[EXTRACTION].extraction_chain_p = extraction_chain
# print(app.nodes[EXTRACTION].__)
# app.bind(extraction_chain_p = extraction_chain)




In [30]:
# def create_inner_test(p: str):
#   pp=p+p
#   def get_pp(x: str):
#     return pp+x
#   return get_pp

# func = create_inner_test('test')

# # func = create_symbol_extraction_node(llm)
# print(func('who'))


testtestwho


In [56]:
app = create_workflow(llm)

create_get_route_node
create_symbol_extraction_node
create_symbol_extraction_node
create_chain_node


In [58]:
# request = "tell me about Apple?"
request = "tell me about Uber?"
# app.debug = True
# extraction_chain = create_extraction_chain(llm)
# chat_chain = create_chat_chain(llm)
# result = app.invoke(input = {"request": request},
result = app.invoke(input = {"request": request}, config={"configurable": {"thread_id": "1"}})
# result = app.invoke(input = {"request": request, 'extraction_chain': extraction_chain, 'chat_chain':chat_chain},
    # config={"configurable": {"thread_id": "1"}})
print('Final result:',result['final_answer'])
# result[]
# print(app.invoke(input = {"request": request}))

router_node
State {'request': 'tell me about Uber?'}
request_category: report
where_to
State {'request_category': 'report', 'request': 'tell me about Uber?'}
extraction_node
State {'request': 'tell me about Uber?', 'request_category': 'report'}
is_there_symbol
State {'symbol': 'UBER', 'request': 'tell me about Uber?', 'request_category': 'report'}
get_income_statement_node
Symbol: UBER
get_company_financials_node
Symbol: UBER
get_stock_price_node
Symbol: UBER
final_answer_node
State {'symbol': 'UBER', 'request': 'tell me about Uber?', 'income_statement': '\n    ## Income Statement (as of 2023-12-31)\n    - **Revenue**: $ 37281000000.00\n    - **Gross Profit**: $ 14824000000.00\n    - **Net Income**: $ 1887000000.00\n    - **EBITDA**: $ 2219000000.00\n    - **EPS**:  0.93\n    - **EPS (Diluted)**:  0.87\n    ', 'company_financials': '\n\n      ## Company Overview\n      - **Name**: Uber Technologies, Inc.\n      - **Symbol**: UBER\n      - **Market Capitalization**: 153893855000.0\n    

In [59]:
request = "what Uber's stock price?"
result = app.invoke(input = {"request": request}, config={"configurable": {"thread_id": "2"}})
print('Final result:',result['final_answer'])


router_node
State {'request': "what Uber's stock price?"}
request_category: stock_price
where_to
State {'request_category': 'stock_price', 'request': "what Uber's stock price?"}
extraction_node
State {'request': "what Uber's stock price?", 'request_category': 'stock_price'}
where_to_alone
State {'symbol': 'UBER', 'request': "what Uber's stock price?", 'request_category': 'stock_price'}
Routing to stock_price
get_stock_price_node
Symbol: UBER
final_answer_node
State {'symbol': 'UBER', 'request': "what Uber's stock price?", 'stock_price': '\n      ## Stock Price Information\n    - **Current Price**: $ 73.25\n    - **Volume**:  30557430.00\n    - **50-Day Average Price**: $ 75.38\n    - **200-Day Average Price**: $ 71.99\n    - **EPS**:  2.03\n    - **PE Ratio**:  36.08\n    - **Earnings Announcement**: 2024-10-31 04:00:00+00:00\n    ', 'request_category': 'stock_price'}
Final result: # stock_price for (UBER) 

      ## Stock Price Information
    - **Current Price**: $ 73.25
    - **Volu

In [60]:
request = "do the same for lyft"
result = app.invoke(input = {"request": request}, config={"configurable": {"thread_id": "5"}})
print('Final result:',result['final_answer'])


router_node
State {'request': 'do the same for lyft'}
request_category: company_financials
where_to
State {'request_category': 'company_financials', 'request': 'do the same for lyft'}
extraction_node
State {'request': 'do the same for lyft', 'request_category': 'company_financials'}
where_to_alone
State {'symbol': 'LYFT', 'request': 'do the same for lyft', 'request_category': 'company_financials'}
Routing to company_financials
get_company_financials_node
Symbol: LYFT
final_answer_node
State {'symbol': 'LYFT', 'request': 'do the same for lyft', 'company_financials': '\n\n      ## Company Overview\n      - **Name**: Lyft, Inc.\n      - **Symbol**: LYFT\n      - **Market Capitalization**: 5413931960.0\n      - **Industry**: Software - Application\n      - **Sector**: Technology\n      - **Website**: [https://www.lyft.com](https://www.lyft.com)\n      - **Beta**:  2.049\n      - **Current Price**: $ 13.48\n      ', 'request_category': 'company_financials'}
Final result: # company_financial

In [61]:
request = "I meant stock price"
result = app.invoke(input = {"request": request}, config={"configurable": {"thread_id": "6"}})
print('Final result:',result['final_answer'])


router_node
State {'symbol': 'UNKNOWN', 'request': 'I meant stock price', 'final_answer': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    ', 'request_category': 'stock_price', 'error': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    '}
request_category: stock_price
where_to
State {'request_category': 'stock_price', 'symbol': 'UNKNOWN', 'request': 'I meant stock price', 'final_answer': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    ', 'error': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    '}
extraction_node
State {'symbol': 'UNKNOWN', 'request': 'I meant stock price', 'final_answer': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    ', 'request_category': 'stock_price', 'error': '\n    Unknown Symbol: UNKNOWN\n    Can not produce report for this symbol.\n    '}
where_to_alone
State {'symbol': 'UNKNOWN', 'request': 'I meant 

In [ ]:
request = "Hello, how are you doing?"
result = app.invoke(input = {"request": request}, config={"configurable": {"thread_id": "4"}})
print('Final result:',result['final_answer'])
